## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pycaret.regression  import *


# feature selection
from sklearn.linear_model import LinearRegression
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

#Scailing
from sklearn.preprocessing import PowerTransformer

#Label Encoding
from sklearn.preprocessing import LabelEncoder

In [2]:
#삡운드
import winsound as sd
# 삡싸운드
def beepsound():
    fr = 2000   # range : 37 ~ 32767
    du = 1000 # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

## data 불러오기

In [3]:
# 데이터가 저장된 경로 설정
data_path = '../data/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'train_df.csv', encoding='cp949')
test = pd.read_csv(data_path + 'test_df.csv', encoding='cp949')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
train

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3
...,...,...,...,...,...,...
31995,31995,5011001060063300,2635000026053400,식품,농산물,6
31996,31996,5011001095042400,1168000017002200,식품,기타식품,5
31997,31997,5011001108036200,4119700008012100,식품,농산물,9
31998,31998,5011001115011400,1132000015085100,식품,농산물,3


### 전처리

In [5]:
data = pd.concat([train,test]).reset_index(drop=True);data

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3.0
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3.0
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9.0
3,3,1154500002014200,5011000315087400,식품,농산물,10.0
4,4,1165000021008300,5011000177051200,식품,가공식품,3.0
...,...,...,...,...,...,...
36635,36635,5013000858004400,4725000719072200,식품,농산물,NaN
36636,36636,5013000870018300,2826000106075300,식품,농산물,NaN
36637,36637,5013000897086300,4311100034004300,식품,농산물,NaN
36638,36638,5013000902065100,4145000013011200,식품,농산물,NaN


- Make Feature

In [6]:
# 송하인 기준으로 총 몇개의 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
groupby_SEND_SPG_INNB_sum = data.groupby('SEND_SPG_INNB').sum()['INVC_CONT'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_SEND_SPG_INNB_sum_dix = {x:y for x,y in zip(groupby_SEND_SPG_INNB_sum['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_sum['INVC_CONT'])}
# 피처 생성
data['총물류량'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_sum_dix[x])

In [7]:
# 송하인 기준으로 총 몇명의 수하인에게 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
groupby_SEND_SPG_INNB_count = data.groupby('SEND_SPG_INNB').count()['REC_SPG_INNB'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_SEND_SPG_INNB_count_dix = {x:y for x,y in zip(groupby_SEND_SPG_INNB_count['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_count['REC_SPG_INNB'])}
# 피처 생성
data['거래수하인수'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_count_dix[x])

In [8]:
# 평균 한건당 몇번의 수화물을 보내는지에 대한 피처 생성
data['평균거래량'] = data['총물류량'] / data['거래수하인수']

In [9]:
dic_max = {x: y for x, y in zip(data.groupby('SEND_SPG_INNB').max()[['DL_GD_LCLS_NM', '거래수하인수']].query('거래수하인수 > 100').reset_index()['SEND_SPG_INNB'], 
                     data.groupby('SEND_SPG_INNB').max()[['DL_GD_LCLS_NM', '거래수하인수']].query('거래수하인수 > 100').reset_index()['DL_GD_LCLS_NM'])}
data['최대거래품목'] = data['SEND_SPG_INNB'].apply(lambda x: dic_max[x] if x in dic_max else '거래량적음')

In [10]:
# 인코딩과 스케일링을 위함
data_index = data['index']
y_data = data['INVC_CONT']

data = data.drop(columns=['index', 'INVC_CONT'])

- 결측치 채움 -> 없음

- Encoding, Scailing

In [11]:
# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

In [12]:
num_features = num_features[2:];num_features

['총물류량', '거래수하인수', '평균거래량']

In [13]:
#수치형변수 scailing
data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

#라벨인코딩
#위의 범주형 변수들을 라벨인코딩
for f in cat_features:
    data[f] = LabelEncoder().fit_transform(data[f])
# # 원핫인코딩
# data = pd.get_dummies(data)

In [14]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_index, data],axis=1)
data = pd.concat([data, y_data],axis=1)

- Data split

In [15]:
#train과 test를 분리
train = data.iloc[:32000,:]
test = data.iloc[32000:,:]
test = test.drop(columns = ['INVC_CONT']).reset_index(drop=True)

## Modeling with pycaret

- Model setting

In [17]:
# #pycaret에 넣도록 변형
# train = pd.concat([pd.DataFrame(train),y_train],axis=1)
# test = pd.concat([test_index, pd.DataFrame(test)],axis=1)

In [18]:
train = train.iloc[:,1:]

In [19]:
reg = setup(train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = 'INVC_CONT', # 목표 변수는 매칭성공여부이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = True, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
#             numeric_features=['전체회차','현재회차'], # 숫자로써의 의미가 있다.
            session_id = 42,
            fold_shuffle = True
            )

,Description,Value
0,session_id,42
1,Target,INVC_CONT
2,Original Data,"(32000, 9)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,0
6,Transformed Train Set,"(31967, 8)"
7,Transformed Test Set,"(33, 8)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [20]:
top5 = compare_models(sort='RMSE', fold=5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1.8448,24.1467,4.9103,0.2653,0.3610,0.3500,1.2960
lightgbm,Light Gradient Boosting Machine,1.8710,24.5571,4.9540,0.2561,0.3675,0.3531,0.8080
lar,Least Angle Regression,2.0995,26.1680,5.1121,0.2091,0.4126,0.4232,0.0120
ridge,Ridge Regression,2.0930,26.1681,5.1121,0.2091,0.4110,0.4213,0.0120
omp,Orthogonal Matching Pursuit,2.0249,26.3846,5.1330,0.2027,0.3998,0.3961,0.0100
br,Bayesian Ridge,2.1511,26.4462,5.1393,0.2007,0.4209,0.4402,0.0140
catboost,CatBoost Regressor,1.8827,26.5337,5.1490,0.1911,0.3688,0.3549,3.9240
lr,Linear Regression,2.1366,26.6042,5.1545,0.1960,0.4180,0.4338,0.0100
lasso,Lasso Regression,1.9752,27.8334,5.2714,0.1595,0.3702,0.3802,0.1960
en,Elastic Net,1.9975,28.4086,5.3257,0.1421,0.3711,0.3846,0.2340


- Hyperparameter tuning

In [21]:
#catboost
catboost = create_model('catboost')
#hyperparameter tuning
catboost = tune_model(catboost, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.8764,25.0186,5.0019,0.1683,0.3659,0.3601
1,1.8213,20.4941,4.5270,0.3437,0.3543,0.3577
2,1.8656,23.0427,4.8003,0.3460,0.3617,0.3536
3,1.8186,24.4682,4.9465,0.3385,0.3569,0.3429
4,1.8372,23.5267,4.8504,0.2651,0.3546,0.3483
Mean,1.8438,23.3101,4.8252,0.2923,0.3587,0.3525
SD,0.0233,1.5690,0.1650,0.0690,0.0045,0.0062


In [22]:
beepsound()

- Fit & predict

In [23]:
model = catboost
model.fit(train.iloc[:,:-1],train['INVC_CONT'])
pred = model.predict(test.iloc[:,1:])

In [24]:
submission['INVC_CONT'] = pred

### Make submission 

In [25]:
t = pd.Timestamp.now()
fname = f'submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv'
# fname = 'submission_etpercentile.csv'
submission.to_csv('../submission/'+fname, index=False)
print("'{}' is ready to submit." .format(fname))

'submission_12142306.csv' is ready to submit.


In [24]:
beepsound()

# END